<a href="https://colab.research.google.com/github/yanii/aicompute/blob/main/AI_Efficiency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Based on https://openai.com/blog/ai-and-efficiency/

#Data

In [ ]:
data = [
    {'name': 'AlexNet', 'perf':'AlexNet', 'tflopsdays': 3.1, 'date': '01/04/2012'},
    {'name': 'GoogLeNet', 'perf':'AlexNet', 'tflopsdays': 0.71, 'date': '17/09/2014'},
    {'name': '*GoogLeNet-lowrank', 'perf':'AlexNet', 'tflopsdays': 0.5183, 'date': '20/11/2015'},
    {'name': 'MobileNet', 'perf':'AlexNet', 'tflopsdays': 0.28, 'date': '17/04/2017'},
    # {'name': 'ShuffleNet (1x)', 'perf':'AlexNet', 'tflopsdays': 0.15, 'date': '03/07/2017'},
    {'name': 'ShuffleNet v2 (1x)', 'perf':'AlexNet', 'tflopsdays': 0.15, 'date': '03/07/2018'},
    {'name': 'EfficientNet-b0', 'perf':'AlexNet', 'tflopsdays': 0.069, 'date': '28/05/2019'},
    {'name': 'ResNet-50', 'perf':'ResNet-50', 'tflopsdays': 17, 'date': '10/06/2015'},
    # {'name': 'SqueezeNet', 'perf':'AlexNet', 'tflopsdays': 0.73, 'date': '24/02/2016'},
    # {'name': '*Root-8', 'perf':'ResNet-50', 'tflopsdays': 12.6, 'date': '20/05/2016'},
    {'name': '*Root-64', 'perf':'ResNet-50', 'tflopsdays': 9.29, 'date': '20/05/2016'},
    {'name': 'EfficientNet-b1', 'perf':'ResNet-50', 'tflopsdays': 0.75, 'date': '28/05/2019'},
]

In [ ]:
import pandas as pd

In [ ]:
dataframe = pd.DataFrame.from_dict(data)
dataframe['date'] = pd.to_datetime(dataframe['date'], format='%d/%m/%Y') 
dataframe = dataframe.sort_values(by=['date']) 

In [ ]:
dataframe

,name,perf,tflopsdays,date
0,AlexNet,AlexNet,3.1000,2012-04-01
1,GoogLeNet,AlexNet,0.7100,2014-09-17
6,ResNet-50,ResNet-50,17.0000,2015-06-10
2,*GoogLeNet-lowrank,AlexNet,0.5183,2015-11-20
7,*Root-64,ResNet-50,9.2900,2016-05-20
3,MobileNet,AlexNet,0.2800,2017-04-17
4,ShuffleNet v2 (1x),AlexNet,0.1500,2018-07-03
5,EfficientNet-b0,AlexNet,0.0690,2019-05-28
8,EfficientNet-b1,ResNet-50,0.7500,2019-05-28


# Altair Plot

In [ ]:
import altair as alt
from altair import datum

In [ ]:
points=alt.Chart(dataframe, width=320, height=240).encode(
    alt.X('date:T', title='Publication Date', scale=alt.Scale(domain=[pd.to_datetime('1/1/2012', format='%d/%m/%Y'), pd.to_datetime('1/7/2021', format='%d/%m/%Y')])),
    alt.Y('tflopsdays:Q', scale=alt.Scale(type='log', base=10, zero=False), axis=alt.Axis(format='.0e'), title='Training Cost (Teraflops/s-days)'),
    alt.Color('perf:N', legend=alt.Legend(orient='top-left', offset=5, 
                                          fillColor='white', padding=5, 
                                          strokeColor='grey',
                                          direction='horizontal',
                                          title='ImageNet Accuracy',
                                          titleBaseline='middle',
                                          titleOrient='left'),
              title='DNN Architecture'),
    text='name:N'
).mark_circle(size=100)

annotations_alexnet = points.transform_filter(
    datum.perf == 'AlexNet'
).mark_text(
    align='left',
    baseline='middle',
    fontSize = 10,
    dx = 7,
)

annotations_resnet50 = points.transform_filter(
    datum.perf == 'ResNet-50'
).mark_text(
    align='left',
    baseline='middle',
    fontSize = 10,
    dx = 7,
)

chart = (points+annotations_alexnet+annotations_resnet50).configure_axis(
    gridColor='lightgrey',
    gridOpacity=0.33,
    labelFontStyle='Roboto-Regular',
    titleFont='Roboto-Regular',
)

chart.interactive()

alt.LayerChart(...)

In [163]:
#data_years=pd.DataFrame({'year': pd.to_datetime(dataframe['date'].dt.year.unique(), format='%Y')})
data_years=pd.DataFrame({'year': pd.to_datetime(range(2012, 2022), format='%Y'), 'label': ['Moore\'s Law']+['',]*(len(range(2012,2022))-1)})

In [164]:
moores = alt.Chart(data_years, width=320, height=240).transform_calculate(
    moore='1/pow(2, (year(datum.year) - 1972)/2)',
).encode(
    x=alt.X('year:T', title='Year'),
    y=alt.Y('moore:Q', axis=alt.Axis(format='.0e'), title='CPU Transistors'),
).mark_line(size=2, color='gray', strokeDash=[5])

moores_label = moores.encode(
    x='max(year):T',
    y=alt.Y('moore:Q'),
    text='label',
).mark_text(align='right', dx=-10, dy=-7, angle=0, color='gray',)

chart = (moores+moores_label).configure_axis(
    gridColor='lightgrey',
    gridOpacity=0.33,
    labelFont='Roboto-Regular',
    titleFont='Roboto-Regular',
)

chart.interactive()

alt.LayerChart(...)

In [181]:
base=alt.Chart(dataframe, width=320, height=240).transform_calculate(
    costpercent="datum.tflopsdays / 3.1000",
).encode(
    alt.X('date:T', title='Date', scale=alt.Scale(domain=[pd.to_datetime('1/4/2012', format='%d/%m/%Y'), pd.to_datetime('1/7/2021', format='%d/%m/%Y')])),
    alt.Y('costpercent:Q', axis=alt.Axis(format='.0e'), 
          scale=alt.Scale(type='log', base=10, zero=False, padding=22)
          ),
    text='name:N'
)

points=base.encode(
    alt.Color('perf:N', legend=alt.Legend(orient='top-left',
                                          offset=5, fillColor='white',
                                          padding=5, strokeColor='grey',
                                          direction='horizontal',
                                          titleBaseline='middle',
                                          titleOrient='left'),
              title='Equiv. ImageNet Accuracy'),
).mark_circle(size=100)

moores = alt.Chart(data_years, width=320, height=240).transform_calculate(
    moore='1/pow(2, (year(datum.year) - 1972)/2)',
).transform_calculate(
    moorepercent='datum.moore/(1/pow(2, (2012 - 1972)/2))'
).encode(
    alt.X('year:T', title='Date'),
    alt.Y('moorepercent:Q', 
          scale=alt.Scale(type='log', base=10, zero=False), 
          axis=alt.Axis(format='.0e'), title='Training Cost Decrease Factor'),
    text='label',
).mark_line(size=2, color='lightgray', strokeDash=[5])

moores_label = moores.mark_text(align='right', color='gray', dx=90, dy=10, angle=15)

annotations_alexnet = points.transform_filter(
    datum.perf == 'AlexNet'
).mark_text(
    align='left',
    baseline='middle',
    fontSize = 10,
    dx = 7,
)

annotations_resnet50 = points.transform_filter(
    datum.perf == 'ResNet-50'
).mark_text(
    align='left',
    baseline='middle',
    fontSize = 10,
    dx = 7,
)

chart = (
    moores+moores_label+points+annotations_alexnet+annotations_resnet50
).resolve_scale(y='shared').configure_axis(
    gridColor='lightgrey',
    gridOpacity=0.33,
    labelFont='Roboto-Regular',
    titleFont='Roboto-Regular',
)

chart.interactive()

alt.LayerChart(...)